In [167]:
from transformers import pipeline

nasa_smd_pipeline = pipeline("question-answering", model="nasa-impact/nasa-smd-ibm-v0.1")

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at nasa-impact/nasa-smd-ibm-v0.1 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [168]:
astroBERT_pipeline = pipeline("question-answering", model="adsabs/astroBERT")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at adsabs/astroBERT and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [179]:
from datasets import load_dataset

squad_datasets = load_dataset("squad")
squad_data = squad_datasets["train"][50000:70000]

Using the latest cached version of the module from /Users/rajdangol/.cache/huggingface/modules/datasets_modules/datasets/squad/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453 (last modified on Fri Dec 15 15:54:42 2023) since it couldn't be found locally at squad., or remotely on the Hugging Face Hub.


In [180]:
from evaluate import load
squad_metric = load("squad")

In [181]:
# nasa_result = nasa_smd_pipeline([dict(context="KU's School of Business launched interdisciplinary management science graduate studies in operations research during Fall Semester 1965. The program provided the foundation for decision science applications supporting NASA Project Apollo Command Capsule Recovery Operations.", question="What NASA program benefited from the use of decision science applications?"),
#                                  dict(context="On October 21, 1959, Eisenhower approved the transfer of the Army's remaining space-related activities to NASA. On July 1, 1960, the Redstone Arsenal became NASA's George C. Marshall Space Flight Center, with von Braun as its first director. Development of the Saturn rocket family, which when mature, would finally give the US parity with the Soviets in terms of lifting capability, was thus transferred to NASA.", question="The Redstone Arsenal became the George C. Marshall Space Flight Center when?")])

# nasa_result

In [182]:
# Converts squad data to squad reference data format
def convert_to_squad_reference_format(dataset):
    reference_result = []
    for i in range(len(dataset["context"])):
        reference_item = {
            "id": dataset["id"][i],
            "answers": [{"text": ans, "answer_start": start} for ans, start in zip(dataset["answers"][i]["text"], dataset["answers"][i]["answer_start"])]
        }
        reference_result.append(reference_item)
    return reference_result


In [190]:
reference_data = convert_to_squad_reference_format(squad_data)
# reference_data

In [184]:
len(reference_data)

20000

In [185]:
# Converts squad dataset to question-context format
def convert_to_question_context_format(dataset):
    prediction_result = []
    for i in range(len(dataset["context"])):
        item = {
            "context": dataset["context"][i],
            "question": dataset["question"][i],
            # "id": dataset["id"][i],
            # "title": dataset["title"][i],
            # "answers": [{"text": ans, "answer_start": start} for ans, start in zip(dataset["answers"][i]["text"], dataset["answers"][i]["answer_start"])]
        }
        prediction_result.append(item)
    return prediction_result

In [186]:
# Converts pipeline's answer to squad prediction format
def convert_to_prediction_format(prediction):
    prediction_result = []
    for item in prediction:
        prediction_item = {
            "prediction_text": item["answer"],
            "id": item["id"]
        }
        prediction_result.append(prediction_item)
    return prediction_result


In [187]:
def apply_pipeline(pipeline, data):
    data = convert_to_question_context_format(data)
    prediction = pipeline(data)
    for i,item in enumerate(prediction):
        item['id'] = squad_data['id'][i]
    prediction_result = convert_to_prediction_format(prediction)

    return squad_metric.compute(predictions=prediction_result, references=reference_data)

In [188]:
# Results from nasa-smd pipeline
nasa_smd_result = apply_pipeline(nasa_smd_pipeline, squad_data)
nasa_smd_result

{'exact_match': 0.12, 'f1': 4.92795707620676}

In [189]:
astroBERT_result = apply_pipeline(astroBERT_pipeline, squad_data)
astroBERT_result

{'exact_match': 0.45, 'f1': 4.977317915473823}